# Open Street Maps Data Import and Analysis

# Intro

Data for Raleigh, North Carolina was downloaded from MapZen (https://mapzen.com/data/metro-extracts).

It was converted to JSON while being structured and corrected/normalized, then imported into mongodb.

In [1]:
from pymongo import MongoClient
client = MongoClient()
db = client.osm
collection = db.raleigh

# Problems encountered

There are certainly multiple problems or areas for improvement, but three issues that stuck out were:

1. Inconsistent Street Names
2. (Likely) Duplicate Nodes
3. Questionable Validity of Education Ammenity Tags

### Street Names

Aside from the expected standardization (such as "st" to "Street") that was required, some nodes had street names that were easily correctible but very specific.  This means they required manual checking (for example, consistency with Google Maps) and specific one-off corrections:

* "Meadowmont Village CIrcle" becomes "Meadowmont Village Circle"
* "LaurelcherryStreet" becomes "Laurel Cherry Street"
* "Garrett Driver" becomes "Garrett Drive"

Others had street names that could not be verified:

* "Triangle Family Practice"
*  Multiple similar names with no best choice:
    * "NC Highway 55 West"
    * "NC Highway 55"
    * "Highway West"
    * "Highway 55 West"
    * "Highway 55"
    * "US 55"


### Duplicate Nodes

Many nodes had duplicate entries- separate node records that differed only in the created timestamp, id, and ObjectId.  These seem to have been imported using automated methods since the user is the same for the duplicate records and the timestamp is either the same or very close (within 5 minutes).  

These could likely be merged in order to reduce the size of the database without losing meaninful information as long as the 'way' records are also checked for references to these nodes.

Sample code for listing one of these sets:

In [2]:
pipeline = [
    {'$group':{'_id':'$pos', 'count':{'$sum':1}}},
    {'$sort':{'count':-1}},
    {'$skip':1}, #skip documents with no position
    {'$limit':1}
]
documents = collection.aggregate(pipeline, allowDiskUse=True)

for r in documents['result']:
    pos = r['_id']
    print("-"*30,"\n",pos,"\n","-"*30,"\n")
    overlaps = collection.aggregate({'$match':{'pos':pos}})
    for o in overlaps['result']:
        print(o)

------------------------------ 
 [35.7877479, -78.8708045] 
 ------------------------------ 

{'_id': ObjectId('559c8cfc77608123d09ec37b'), 'id': '195496249', 'pos': [35.7877479, -78.8708045], 'type': 'node', 'created': {'timestamp': '2013-10-04T18:14:14Z', 'version': '7', 'uid': '1494110', 'user': 'KristenK', 'changeset': '18183244'}}
{'_id': ObjectId('559c8d2d77608123d0c1d47b'), 'id': '2482640859', 'pos': [35.7877479, -78.8708045], 'type': 'node', 'created': {'timestamp': '2013-10-04T18:13:55Z', 'version': '1', 'changeset': '18183244', 'user': 'KristenK', 'uid': '1494110'}}
{'_id': ObjectId('559c8d2d77608123d0c1d47c'), 'id': '2482640856', 'pos': [35.7877479, -78.8708045], 'type': 'node', 'created': {'timestamp': '2013-10-04T18:13:55Z', 'version': '1', 'changeset': '18183244', 'user': 'KristenK', 'uid': '1494110'}}
{'_id': ObjectId('559c8d2d77608123d0c1d47f'), 'id': '2482640858', 'pos': [35.7877479, -78.8708045], 'type': 'node', 'created': {'timestamp': '2013-10-04T18:13:55Z', 'versio

### Use of Amenity Tags for Education

After importing the data I took a look at a few of the "education" amenity tags ('university', 'college', and 'school') to see how the major colleges and universities in the area may be marked.

In [3]:
#Example Code: List of 'university' tags
pipeline = [
    {'$match':{'amenity':'university'}},
    {'$group':{'_id':'$name', 'count':{'$sum':1}}},
    {'$sort':{'count':-1}},
]
documents = collection.aggregate(pipeline)
for r in documents['result']:
    print(r)

{'_id': 'Duke University East Campus', 'count': 3}
{'_id': 'Duke University Medical Center', 'count': 1}
{'_id': 'Campbell University: Norman Adrian Wiggins School of Law', 'count': 1}
{'_id': "St. Augustine's University", 'count': 1}
{'_id': 'Duke University Central Campus', 'count': 1}
{'_id': 'JC Raulston Arboretum at NC State University', 'count': 1}
{'_id': 'North Carolina Central University', 'count': 1}
{'_id': 'Duke University West Campus', 'count': 1}
{'_id': 'North Carolina State University (Centennial Campus)', 'count': 1}
{'_id': None, 'count': 1}
{'_id': 'William Peace University', 'count': 1}
{'_id': 'Campbell University RTP Campus', 'count': 1}


These tags seem to be infrequently used and are inconsistent when they are used.  According to the specificaitons (https://wiki.openstreetmap.org/wiki/Map_Features#Education):

* "university" indicates a university campus.
* "college" indicates a college campus or building
* "school" indicates a school and grounds.

While some tags are as expected ("Duke University East Campus" appears multiple times as "university"), there are issues.

* There is no name for 1 universitiy, 28 college, and 21 school entries.
* "Durham Tech Community College" appears as a "college" while "Durham Technical Community College" appears twice as a school.
* "Duke University" is listed as a "school" while various campus regions are listed as "university".

I believe that this inconsistency is at least partially to blame on the unclear documentation for these tags.

# Overview of the Data

Filesizes:

* XML: 486.2 MiB
* JSON: 562.6 MiB

###Summary info about imported data

In [4]:
total = collection.find().count()
total_nodes = collection.find({"type":"node"}).count()
total_ways = collection.find({"type":"way"}).count()

print("{:,} Total Records".format(total))
print("{:,} Nodes".format(total_nodes))
print("{:,} Ways".format(total_ways))

2,735,730 Total Records
2,524,259 Nodes
211,464 Ways


###Total Users and Top Users (by number of records)

In [5]:
pipeline = [
    {'$group':{'_id':'$created.user', 'count':{'$sum':1}}},
    {'$sort':{'count':-1}},
]
users = collection.aggregate(pipeline)['result']

print('Total Users: {:,}'.format(len(users)))
print('Top Users by number of records:')
for u in users[0:5]:
    print(u)

Total Users: 696
Top Users by number of records:
{'_id': 'jumbanho', 'count': 2139329}
{'_id': 'woodpeck_fixbot', 'count': 128144}
{'_id': 'yotann', 'count': 67765}
{'_id': 'JMDeMai', 'count': 63378}
{'_id': 'runbananas', 'count': 43244}


### GNIS and TIGER Data
Some data was imported into OSM from GNIS (USGS Geographic Names Information System) and TIGER (Topologically Integrated Geographic Encoding and Referencing system).  However, this is a small fraction of the data overall:

In [6]:
#nodes with gnis
nodes_with = collection.find({'gnis':{'$exists':True}}).count()
nodes_without = collection.find({'gnis':{'$exists':False}}).count()
print("Fraction of Nodes with GNIS Data = {:.2%}".format(nodes_with/(nodes_with+nodes_without)))

#ways with tiger
ways_with = collection.find({'tiger':{'$exists':True}}).count()
ways_without = collection.find({'tiger':{'$exists':False}}).count()
print("Fraction of Ways with TIGER Data = {:.2%}".format(ways_with/(ways_with+ways_without)))

Fraction of Nodes with GNIS Data = 0.03%
Fraction of Ways with TIGER Data = 0.65%


# Additional Ideas

How has the dataset built up over time?  Using the timestamp value it is possible to count the number of nodes or ways added over time- in this case by month.  It looks like January 2010 had the highest count for both types, but big months for 'Ways' were mostly in 2013 and big months for 'Nodes' were mostly in 2009.

In [10]:
#Group by first 7 characters of created timestamp
from collections import Counter
def get_monthly_count(collection, doctype):
    """Return the monthly count as a dict based on start of timestamp"""
    pipeline = [
        {'$match':{'type':doctype}},
        {'$group':{'_id':{'$substr':['$created.timestamp',0,7]}, 'count':{'$sum':1}}},
        {'$sort':{'count':-1}}
        ]
    counts = collection.aggregate(pipeline)['result']
    count_dict = Counter()
    for d in counts:
        count_dict[d['_id']] = d['count']
    return count_dict

node_counts = get_monthly_count(collection, 'node')
way_counts = get_monthly_count(collection, 'way')

print('Average Nodes per Month = {:,}'.format(int(sum(node_counts.values())/len(node_counts))))
print('Average Ways per Month = {:,}'.format(int(sum(way_counts.values())/len(way_counts))))

print('\nMonths with the most Nodes:\n','-'*30)
for m in node_counts.most_common(5):
    print("{} with {:,} Nodes".format(m[0],m[1]))
    
print('\nMonths with the most Ways:\n','-'*30)
for m in way_counts.most_common(5):
    print("{} with {:,} Ways".format(m[0],m[1]))

Average Nodes per Month = 28,684
Average Ways per Month = 2,458

Months with the most Nodes:
 ------------------------------
2010-01 with 1,267,448 Nodes
2009-11 with 271,430 Nodes
2009-07 with 144,939 Nodes
2009-08 with 135,390 Nodes
2009-09 with 46,642 Nodes

Months with the most Ways:
 ------------------------------
2010-01 with 89,284 Ways
2013-02 with 20,061 Ways
2013-01 with 7,724 Ways
2009-11 with 7,109 Ways
2013-04 with 5,135 Ways


In [8]:
#Idea 2

In [9]:
#Idea 3

* update time vs location very large scatterplot- subsample?
* Types of data input by users- eg same user did most Ways
* kinds of tags by node, way, relation

# Conclusion